## **Linear Regression**

In [112]:
import numpy as np
import pandas as pd

class MyLineReg:
    def __init__(self, n_iter, learning_rate, weights=None, metric=None, reg=None, l1_coef=0, l2_coef=0):
        self.n_iter = n_iter
        self.learning_rate = learning_rate
        self.weights = weights
        self.metric = metric
        self.best_metric = None
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef
        self.reg = reg
        
    def __str__(self):
        return f"MyLineReg class: \n{' '.join([f'{key} = {value}' for key, value in self.__dict__.items()])}"

    def _metric(self, y_train, y_pred):
        if self.metric == 'mae':
            return (np.abs(y_train - y_pred)).mean()
        if self.metric == 'mse':
            return ((y_train - y_pred) ** 2).mean()
        if self.metric == 'rmse':
            return (((y_train - y_pred) ** 2).mean()) ** 0.5
        if self.metric == 'mape':
            return (np.abs((y_train - y_pred) / y_train) * 100).mean()
        if self.metric == 'r2':
            return 1 - (((y_train - y_pred) ** 2).sum() / ((y_train - y_train.mean()) ** 2).sum())  
            
    def fit(self, X_train, y_train, verbose=False):
        number_of_observations, number_of_features = X_train.shape
        
        X_train = np.hstack([np.ones(number_of_observations).reshape(-1, 1), X_train.values])
        self.weights = np.ones(number_of_features + 1)
        
        y_pred = X_train @ self.weights
        penalty = 0
        gradient_penalty = 0
        
        MSE = ((y_train - y_pred) ** 2).mean() + penalty
        
        if verbose:
            print(f"start | loss: {MSE} | {self.metric}: {_metric(y_train, y_pred)}")

        for i in range(1, self.n_iter+1):
            if self.reg == 'l1':
                penalty = self.l1_coef * np.sum(np.abs(self.weights))
                gradient_penalty = self.l1_coef * np.sign(self.weights)
            elif self.reg == 'l2':
                penalty = self.l2_coef * np.sum((self.weights) ** 2)
                gradient_penalty = 2 * self.l2_coef * self.weights
            elif self.reg == 'elasticnet':
                penalty = self.l1_coef * np.sum(np.abs(self.weights)) + self.l2_coef * np.sum((self.weights) ** 2)
                gradient_penalty = self.l1_coef * np.sign(self.weights) + 2 * self.l2_coef * self.weights
                
            y_pred = X_train @ self.weights
            MSE = ((y_train - y_pred) ** 2).mean() + penalty
            gradient = 2 / number_of_observations * X_train.T @ (y_pred - y_train) + gradient_penalty
            if callable(self.learning_rate):
                learning_rate_temp = self.learning_rate(i)
                self.weights -= learning_rate_temp * gradient
            else:
                self.weights -= self.learning_rate * gradient
            
            if verbose and i % verbose == 0:
                print(f" {i} | loss: {MSE} | {self.metric}: {_metric(y_train, y_pred)}")
        y_pred = X_train @ self.weights        
        self.best_metric = self._metric(y_train, y_pred)
        
    def get_coef(self):
        return self.weights[1:]

    def predict(self, X_test):
        number_of_observations, number_of_features = X_test.shape
        X_test = np.hstack([np.ones(number_of_observations).reshape(-1, 1), X_test.values])
        
        return X_test @ self.weights

    def get_best_score(self):
        return self.best_metric